In [1]:
'''Testing for STATIC DATASET RL IS IT WORK??????'''
'''RL for Classification'''
import math
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Bernoulli
from torch.autograd import Variable
from itertools import count
import matplotlib.pyplot as plt
import random
import numpy as np
import gym
from IPython.display import clear_output



## Credit Card Data를 이용
train = pd.read_csv("./uci_creditcard-train-0.0-0.0.csv")
test = pd.read_csv("./uci_creditcard-test-0.0-0.0.csv")

train = train.to_numpy()
test = test.to_numpy()

plt.ion()



In [2]:
'''열별로 데이터 전처리하는 과정 '''
def z_score_normalize(data) :
    normal_data = data
    for columns in range(data.shape[1]-2) :
        normal_data[:][columns] = (data[:][columns] - np.mean(data[:][columns])) /\
                                    np.std(data[:][columns])

    return normal_data


normalized_train = z_score_normalize(train)
normalized_test = z_score_normalize(test)

In [3]:
## Experience Replay
## 학습에 이용할 메모리
class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, transition):
        """transition 저장"""
        if len(self.memory) < self.capacity:
            self.memory.append(transition)
        self.position = (self.position + 1) % self.capacity
        if len(self.memory) >= self.capacity: 
            self.memory[self.position] = transition

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

In [4]:
## 큰 차원의 데이터를 다루기위해서 사용


## 학습에 쓰이는  NN 
## Input은 State가 되고 Output은 Q - value가 된다.
class NN(nn.Module):

    def __init__(self, input_size, hidden_size, action_size):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size,hidden_size) 
        self.fc2 = nn.Linear(hidden_size,hidden_size*2)
        self.fc3 = nn.Linear(hidden_size*2,hidden_size)
        self.fc4 = nn.Linear(hidden_size,1) ## Classification에서 0을 선택할 확률이 OUTPUT 
       
        
    ## 최적화 중에 다음 행동을 결정하기 위해서 하나의 요소 또는 배치를 이용해 호촐됨.
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x)) 
        x = F.relu(self.fc3(x))
        ## 마지막 결과값이 확률로 나오게 만듬 . 
        x = torch.sigmoid(self.fc4(x))

        return x
      

In [5]:
## Hyperparameter
BATCH_SIZE = 100
hidden_size = 100
train_num = 5000

## Input data size는 Column의 개수 -1 마지막 두개의 열은 Classification과 Valide Set을 구분.
input_size = train.shape[1]-3
## Classification 이므로 Action은 2가 나옴.
action_size = 2

## 큰 차원의 데이터를 처리하기 위해서 NN을 사용
policy_net = NN(input_size,hidden_size, action_size)
target_net = NN(input_size,hidden_size, action_size)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

## 최적화 기준 RMS(Root Mean Squeare)
optimizer = optim.RMSprop(policy_net.parameters(), lr =0.001)



In [6]:
## Reward를 받을 때 데이터셋의 비율이 다르므로 Reward도 다르게 적용
def get_reward(action,target) :
    if action == target :
        accuracy = 1
        if action == 1 : 
            reward = 23996 / 5356
        elif action == 0 :
            reward = 23996 / 18640
    else :
        accuracy = 0
        if action == 1 :
            reward = -23996 / 5356
        elif action == 0 :
            reward = - 23996 / 18640
    
    return reward,accuracy


In [7]:
def plot_durations(reward):
    plt.figure(2)
    plt.clf()
    reward = torch.FloatTensor(reward)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    #plt.axhline(y=90,linestyle ='--')
    plt.plot(reward.numpy())
    if len(reward) >= 100:
        means = reward.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  
    
    
    
    

In [8]:
total_accuracy = []


## Traing set을 이용한 학습 
##
for i_num in range(train_num):
    ## Batch size 크기 만큼의 Trajectory가 만들어짐.
    trajectory = normalized_train[np.random.choice(normalized_train.shape[0],BATCH_SIZE,replace =False),:]
    total_acc = 0
    ## Discout factor
    GAMMA = 0.99
    ## Trajectory를 만듬 길이는 BATCH_SIZE
    ## Gradient Descent
    optimizer.zero_grad()
    loss = 0
    total_loss = 0
    ## Trajectory를 따라가면서 학습
    for t in range(BATCH_SIZE) :      
        ## state를 tensor로 바꿔서 이용
        state = trajectory[t][1:24]
        state = torch.FloatTensor(state)
        state = Variable(state)
        ## Classification Target
        target = trajectory[t][24]
        
        ## Action을 선택하기 위해 이항분포 => Discrete Action space
        probs = policy_net(state)
        m = Bernoulli(probs)
        action = m.sample()
        action = action.data.numpy().astype(int)[0]
        ## With Discout reward
        reward,accuracy= get_reward(action,target)
        if t >1 : 
            reward = reward * GAMMA
        total_acc += accuracy
        
        GAMMA = GAMMA * 0.99
        
        loss = -m.log_prob(action) * reward  ## Negtive score function x reward
        total_loss += loss
        
    total_accuracy.append(total_acc)
    loss = total_loss / BATCH_SIZE
    if (i_num+1) % 10 == 0 :
        print('Current_train_num : ',i_num, "LOSS : ",loss.item())
    loss.backward()
    ## Optimize
    optimizer.step()
    
    if (i_num+1) % 100 == 0 :
        plot_durations(total_accuracy)
    if (i_num+1) % 500 ==0 :
        clear_output()
    
    
    
        
target_net.load_state_dict(policy_net.state_dict())

<All keys matched successfully>

In [9]:
total_accuracy = []
test_num = 6004 ## Test set의 행의 크기
total_acc = 0

##Test set을 이용한 확인 
##
for i_num in range(test_num):
    
    state = normalized_test[i_num][1:24]
    
    ## state를 tensor로 바꿔서 이용
    state = torch.FloatTensor(state)
    state = Variable(state)
    ## Classification Target
    target = normalized_test[i_num][24]

    ## Action을 선택하기 위해 이항분포 => Discrete Action space
    probs = target_net(state)
    m = Bernoulli(probs)
    action = m.sample()
    action = action.data.numpy().astype(int)[0]
    _,accuracy= get_reward(action,target)

    total_acc += accuracy   
    total_accuracy.append(total_acc)
    if (i_num+1) % 10 == 0 :
        print('Current_train_num : ',i_num, "Accuracy : ",total_acc / i_num)

    if (i_num+1) % 100 == 0 :
        plot_durations(total_accuracy)
    if (i_num+1) % 500 ==0 :
        clear_output()
    
    

In [10]:
print("FINAL RESULT (Correct / Total Test NUM) :   ", total_acc / i_num  )
print("Total Test Num = ", test_num)

FINAL RESULT (Correct / Total Test NUM) :    0.7837747792770281
Total Test Num =  6004


Comments : PG를 이용해서 Classification 문제를 해결하려고 시도해보았다. 성능은 높게 나오진 않았지만, 정형 데이터에서 강화학습 알고리즘을 이용할 수 있을 것 같다.(정형 데이터에 강화학습 알고리즘을 이용해서 Classification 문제를 해결하는 것이 강화학습이라고 할 수 있을까?)